In [1]:
import os
import re
import supervisely as sly
from mmengine.config import Config
from mmdet3d.apis import LidarDet3DInferencer
from src.tests.extract_weights_url import find_weights_url
from src.sly_utils import download_point_cloud, upload_point_cloud, add_classes_to_project_meta
from src.inference.pcd_inferencer import PcdDet3DInferencer
from src.inference.functional import create_sly_annotation, up_bbox3d, filter_by_confidence
from src.pcd_utils import convert_bin_to_pcd
import json

# To import projects
import os, sys
sys.path.append(os.path.abspath("mmdetection3d"))

# turn off warnings
import warnings
warnings.filterwarnings("ignore")


# globals    
api = sly.Api()
workspace_id = 992
# create project with pointclouds
project_info = api.project.get_or_create(workspace_id, "PointPillars", sly.ProjectType.POINT_CLOUDS)
project_id = project_info.id
dataset_info = api.dataset.get_or_create(project_id, "test")
dataset_id = dataset_info.id
project_meta = sly.ProjectMeta.from_json(api.project.get_meta(project_id))

# pcd_path = "app_data/lyft/LYFT/pointcloud/host-a005_lidar1_1231201454801395736.pcd"
pcd_path = "app_data/lyft/LYFT/pointcloud/host-a005_lidar1_1231201437602160096.pcd"
# pcd_path = "app_data/sly_project/ds0/pointcloud/000021.pcd"  # KITTI
is_bin = False

with open("model_list.json", "r") as f:
    model_list = json.load(f)["detection_3d"]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.8/dist-packages/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


In [17]:
s = sly.nn.inference.Session(api, session_url="http://localhost:8000")
s

In [23]:
sly.json.load_json_file("model_list.json")['detection_3d'][-1]

{'name': 'CenterFormer',
 'paper': 'ECCV',
 'year': 2022,
 'model_name': 'centerformer',
 'base_configs': [],
 'pre_trained_configs': [{'config': 'projects/CenterFormer/configs/centerformer_voxel01_second-attn_secfpn-attn_4xb4-cyclic-20e_waymoD5-3d-3class.py',
   'weights': None,
   'metadata': {'Training Memory (GB)': 14.8},
   'results': [{'Task': '3D Object Detection',
     'Dataset': 'nuScenes',
     'Metrics': {'mAP': 62.7, 'NDS': 68.0}}]}]}

In [21]:
self = s
endpoint = "inference_pointcloud_ids"
url = f"{self._base_url}/{endpoint}"
json_body = self._get_default_json_body()
json_body["state"]["pointcloud_ids"] = [29163585, 29163585]
resp = self._post(url, json=json_body, retries=1)
resp.json()

[[{'class_name': 'car',
   'cuboid_3d': {'sly_id': None,
    'labeler_login': None,
    'updated_at': None,
    'created_at': None,
    'class_id': None,
    '_position': {'_x': 4.566761016845703,
     '_y': -3.0126419067382812,
     '_z': -0.6697812676429749},
    '_rotation': {'_x': 0, '_y': 0, '_z': -2.8183915615081787},
    '_dimensions': {'_x': 4.764708995819092,
     '_y': 1.982856035232544,
     '_z': 1.774356484413147}},
   'score': 0.8461081981658936},
  {'class_name': 'car',
   'cuboid_3d': {'sly_id': None,
    'labeler_login': None,
    'updated_at': None,
    'created_at': None,
    'class_id': None,
    '_position': {'_x': -24.512603759765625,
     '_y': 26.932247161865234,
     '_z': -0.4675849676132202},
    '_rotation': {'_x': 0, '_y': 0, '_z': -1.4695369005203247},
    '_dimensions': {'_x': 4.511403560638428,
     '_y': 1.881795883178711,
     '_z': 1.6634454727172852}},
   'score': 0.8445436358451843},
  {'class_name': 'car',
   'cuboid_3d': {'sly_id': None,
    'labe

In [3]:
pcd_path = "lyft.pcd"

model = model_list[1]
config_info = model["pre_trained_configs"][1]  # or 2
cfg_model = "mmdetection3d/"+config_info["config"]
weights_url = config_info["weights"]
name = model["name"]
print(f"Model: {name} ({cfg_model})")

# Make config
cfg = Config.fromfile(cfg_model)
model_class_names = cfg.class_names
trained_dataset_name = cfg.dataset_type
print(f"Model class names: {model_class_names}")
zero_aux_dims = cfg.dataset_type == "KittiDataset"

# Update project_meta
# project_meta = add_classes_to_project_meta(api, project_meta, project_id, model_class_names)

# Inference
if is_bin:
    inferencer = LidarDet3DInferencer(cfg_model, weights_url, device='cuda:0')
else:
    inferencer = PcdDet3DInferencer(cfg_model, weights_url, device='cuda:0', zero_aux_dims=zero_aux_dims)

results_dict = inferencer(inputs=dict(points=pcd_path), no_save_vis=True)
predictions = results_dict['predictions'][0]
bboxes_3d, labels_3d, scores_3d = predictions['bboxes_3d'], predictions['labels_3d'], predictions['scores_3d']
print(f"Predicted boxes (before filtering): {len(bboxes_3d)}")
bboxes_3d, labels_3d, scores_3d = filter_by_confidence(bboxes_3d, labels_3d, scores_3d, threshold=0.45)
bboxes_3d = [up_bbox3d(bbox3d) for bbox3d in bboxes_3d]
print(f"Predicted boxes: {len(bboxes_3d)}")

# Create annotation
# ann = create_sly_annotation(bboxes_3d, labels_3d, model_class_names, project_meta)
# # Upload pointcloud
# name = f"{name}_{trained_dataset_name}_"+sly.rand_str(4)+".pcd"
# pcd_info = upload_point_cloud(api, dataset_id, pcd_path, name=name)
# # Upload annotation
# pcd_id = pcd_info.id
# api.pointcloud.annotation.append(pcd_id, ann)

# print(f"DONE: {name}")
# print(f"https://dev.supervisely.com/app/point-clouds/?datasetId={dataset_id}&pointCloudId={pcd_id}")

Output()

Model: PointPillars (mmdetection3d/configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-3class.py)
Model class names: ['Pedestrian', 'Cyclist', 'Car']
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


Predicted boxes (before filtering): 29
Predicted boxes: 0


In [ ]:
def visualize_predictions(predictions, conf_thres, pcd_path, name):
    global project_meta, project_id, dataset_id, model_class_names, api
    bboxes_3d, labels_3d, scores_3d = predictions['bboxes_3d'], predictions['labels_3d'], predictions['scores_3d']
    print(f"Predicted boxes (before filtering): {len(bboxes_3d)}")
    bboxes_3d, labels_3d, scores_3d = filter_by_confidence(bboxes_3d, labels_3d, scores_3d, threshold=conf_thres)
    bboxes_3d = [up_bbox3d(bbox3d) for bbox3d in bboxes_3d]
    print(f"Predicted boxes: {len(bboxes_3d)}")

    # Update project_meta
    project_meta = add_classes_to_project_meta(api, project_meta, project_id, model_class_names)
    # Create annotation
    ann = create_sly_annotation(bboxes_3d, labels_3d, model_class_names, project_meta)
    # Upload pointcloud
    pcd_info = upload_point_cloud(api, dataset_id, pcd_path, name=name)
    # Upload annotation
    pcd_id = pcd_info.id
    api.pointcloud.annotation.append(pcd_id, ann)

    print(f"DONE: {name}")
    print(f"https://dev.supervisely.com/app/point-clouds/?datasetId={dataset_id}&pointCloudId={pcd_id}")

In [ ]:
inputs = list(inferencer.preprocess([dict(points=pcd_path_lyft)]))
inputs[0]['inputs']['points'][0][:, -1] = 0.
res = inferencer.forward(inputs[0])
predictions = inferencer.postprocess(res, no_save_pred=True)['predictions'][0]
visualize_predictions(predictions, 0.4, pcd_path_lyft, "pp_kitti_predicts_lyft_3.pcd")

Predicted boxes (before filtering): 12
Predicted boxes: 4
DONE: pp_kitti_predicts_lyft_3.pcd
https://dev.supervisely.com/app/point-clouds/?datasetId=83438&pointCloudId=29162893


In [ ]:
inputs = list(inferencer.preprocess([dict(points=pcd_path_lyft)]))
save_pcd(inputs[0]['inputs']['points'][0].cpu().numpy(), "lyft_filtered.pcd")
upload_point_cloud(api, dataset_id, "lyft_filtered.pcd", name="lyft_filtered_4.pcd")

PointcloudInfo(id=29162918, frame=None, description='', name='lyft_filtered_4.pcd', team_id=None, workspace_id=None, project_id=None, dataset_id=83438, link=None, hash='GiSuzuapscz7GGwAW9/xmt53Ih9pOnzGXn0+A6h/vK4=', path_original='/h5un6l2bnaz1vj8a9qgms4-public/point_clouds/g/J/Uj/4IYh0PG99zDE9v1rZYblXl3jEI3ZEk8v9p3Y4FGpTMjZ84wtQlZsoadCqHaS54B7mrvBeaKA9ZP5BkNrzIoj8XJtHiue8ULt0DjP6cScFChKtwjCTMhumZpYfnqt.pcd', cloud_mime='image/pcd', figures_count=None, objects_count=None, tags=None, meta={}, created_at='2024-01-12T14:15:29.432Z', updated_at='2024-01-12T14:15:29.432Z')

In [ ]:
inputs[0]['inputs']['points'][0].cpu().numpy()[:, 1].max()

15.907563

In [ ]:
res = inferencer.forward(inputs[0])
res = inferencer.postprocess(res, no_save_pred=True)['predictions'][0]
np.mean(res['scores_3d'])

0.18807882687141156

In [ ]:
inferencer.cfg.test_dataloader.dataset.pipeline[1]['transforms'][-1]['point_cloud_range'] = [0, 0, -3, 70.4, 40, 1]

In [ ]:
inferencer._init_pipeline(cfg)

Compose(
    MultiScaleFlipAug3D(transforms=Compose(
    GlobalRotScaleTrans(rot_range=[0, 0], scale_ratio_range=[1.0, 1.0], translation_std=[0, 0, 0], shift_height=False)
    RandomFlip3D(sync_2d=True, flip_ratio_bev_vertical=0.0)
    PointsRangeFilter(point_cloud_range=[0.0, 0.0, -3.0, 70.4000015258789, 40.0, 1.0])
), img_scale=[(1333, 800)], flip=False, pts_scale_ratio=[1.0], flip_direction=['horizontal'])
    Pack3DDetInputs(keys=['points'])(meta_keys=('img_path', 'ori_shape', 'img_shape', 'lidar2img', 'depth2img', 'cam2img', 'pad_shape', 'scale_factor', 'flip', 'pcd_horizontal_flip', 'pcd_vertical_flip', 'box_mode_3d', 'box_type_3d', 'img_norm_cfg', 'num_pts_feats', 'pcd_trans', 'sample_idx', 'pcd_scale_factor', 'pcd_rotation', 'pcd_rotation_angle', 'lidar_path', 'transformation_3d_flow', 'trans_mat', 'affine_aug', 'sweep_img_metas', 'ori_cam2img', 'cam2global', 'crop_offset', 'img_crop_offset', 'resize_img_shape', 'lidar2cam', 'ori_lidar2img', 'num_ref_frames', 'num_views', 'ego2

In [ ]:
from src.inference.pcd_loader import PCDLoader
pcd_path_lyft = "app_data/lyft/LYFT/pointcloud/host-a005_lidar1_1231201437602160096.pcd"
pcd_path_kitti = "app_data/sly_project/ds0/pointcloud/000021.pcd"  # KITTI

loader = PCDLoader(use_dim=5)
loader.transform({"points": pcd_path_kitti})['points'][:20, :3]

LiDARPoints(
    tensor([[65.9970,  3.6680,  2.4440],
        [65.9590,  3.8740,  2.4430],
        [65.6950,  4.0650,  2.4350],
        [50.4730,  3.2770,  1.9190],
        [65.4670,  4.4640,  2.4280],
        [65.5510,  4.5730,  2.4310],
        [77.1930,  5.8780,  2.8270],
        [43.0870,  3.4060,  1.6710],
        [42.4870,  3.4920,  1.6510],
        [42.2620,  3.6070,  1.6430],
        [42.2650,  3.6740,  1.6440],
        [40.1150,  3.6130,  1.5710],
        [39.4170,  3.6750,  1.5480],
        [39.2740,  3.7860,  1.5430],
        [36.3020,  3.6120,  1.4430],
        [35.0410,  3.7080,  1.4010],
        [33.9000,  3.7490,  1.3620],
        [33.8900,  3.8550,  1.3630],
        [33.8880,  3.9630,  1.3630],
        [33.9190,  4.0750,  1.3640]]))

In [ ]:
import numpy as np
from src.pcd_utils import save_pcd
pcd_np_kitti = loader.transform({"points": pcd_path_kitti})['points'].numpy()
pcd_np_lyft = loader.transform({"points": pcd_path_lyft})['points'].numpy()
save_pcd(pcd_np_kitti, "kitti.pcd")
save_pcd(pcd_np_lyft, "lyft.pcd")
pcd_np_mix = np.concatenate([pcd_np_kitti, pcd_np_lyft], axis=0)
save_pcd(pcd_np_mix, "mix.pcd")

# upload
name = "mix.pcd"
pcd_info = upload_point_cloud(api, dataset_id, "mix.pcd", name=name)

name = "kitti.pcd"
pcd_info = upload_point_cloud(api, dataset_id, "kitti.pcd", name=name)

name = "lyft.pcd"
pcd_info = upload_point_cloud(api, dataset_id, "lyft.pcd", name=name)

print(f"https://dev.supervisely.com/app/point-clouds/?datasetId={dataset_id}&pointCloudId={pcd_info.id}")

https://dev.supervisely.com/app/point-clouds/?datasetId=83438&pointCloudId=29162812


In [ ]:
pcd_np_mix[:, 3][:10]
# get avg intensity for kitti
pcd_np_kitti[:, 3].mean()

0.3034299

In [ ]:
# get num points for kitti and  lyft
print(pcd_np_kitti.shape[0])
print(pcd_np_lyft.shape[0])

125260
62364


In [ ]:
cfg.test_dataloader.dataset

{'type': 'NuScenesDataset',
 'data_root': 'data/nuscenes/',
 'ann_file': 'nuscenes_infos_val.pkl',
 'pipeline': [{'type': 'LoadPointsFromFile',
   'coord_type': 'LIDAR',
   'load_dim': 5,
   'use_dim': 5,
   'backend_args': None},
  {'type': 'LoadPointsFromMultiSweeps',
   'sweeps_num': 10,
   'test_mode': True,
   'backend_args': None},
  {'type': 'MultiScaleFlipAug3D',
   'img_scale': (1333, 800),
   'pts_scale_ratio': 1,
   'flip': False,
   'transforms': [{'type': 'GlobalRotScaleTrans',
     'rot_range': [0, 0],
     'scale_ratio_range': [1.0, 1.0],
     'translation_std': [0, 0, 0]},
    {'type': 'RandomFlip3D'},
    {'type': 'PointsRangeFilter',
     'point_cloud_range': [-50, -50, -5, 50, 50, 3]}]},
  {'type': 'Pack3DDetInputs', 'keys': ['points']}],
 'metainfo': {'classes': ['car',
   'truck',
   'trailer',
   'bus',
   'construction_vehicle',
   'bicycle',
   'motorcycle',
   'pedestrian',
   'traffic_cone',
   'barrier']},
 'modality': {'use_lidar': True, 'use_camera': False}